In [26]:
# Для корректной работы ноутбука необходимо установить универсальную среду для запуска решателей minizinc
# Это необходимо для решения задач линейного программирования, описанных в файлах mzn

# Описание на wikipedia
# https://en.wikipedia.org/wiki/MiniZinc

# Документация по установке
# https://docs.minizinc.dev/en/stable/installation.html

# Скачать можно на странице по адресу:
# https://www.minizinc.org/

# Указать путь к minizinc.exe
minizinc_exec_path = "minizinc.exe";

# Путь к решателю АВиГ
bnb_lap_solver_exec_path = "./bin/index.exe";

# Путь к данным решателя по умолчанию
# Для увеличения скорости можно записывать файл в tmpfs или ramdisk (в оперативную память)
# По умолчанию данные для модели записываются в одну директорию с моделью и ноутбуком
default_minizinc_data_dir = "./data_dzn"

# Путь к экземплярам тестов и результам их вычисления
# Директория, где расположены json'ы
default_problem_instances_path = "./data"

async_threads_n = 1

In [2]:
import lib.config

lib.config.global_config["bnb_lap_solver_exec_path"] = bnb_lap_solver_exec_path
lib.config.global_config["minizinc_exec_path"] = minizinc_exec_path
lib.config.global_config["default_problem_instances_path"] = default_problem_instances_path
lib.config.global_config["default_minizinc_data_dir"] = default_minizinc_data_dir

In [3]:
import copy
import json
import time
import datetime
import random
import asyncio
import aiofiles
import aiofiles.os
import pandas as pd
import inspect
from functools import cmp_to_key
from tqdm.notebook import trange, tqdm
from IPython.display import clear_output

In [4]:
import lib.experiments
import lib.utils
import lib.lp
import lib.lap

from lib.utils import (
    mat2d,
    mat3d,
    print_osp_matrix,
    deep_round,
)

from lib.lp import (
    solve_osp_blp,
    solve_lp
)

from lib.lap import (
    solve_lap_scipy,
)

from lib.experiments import (
    generate_test_case_params_twscp,
    create_problem_instance_id,
    write_instance_into_disk,
    create_cost_matrix_twscp,
    calc_obj_from_mat,
    mat2d_lap_sol_into_vec,
    PILStd,
    PILVerbose0,
    solve_osp_lap_bnb_2d_v4_0,
    get_wsrpt_upper_bound,
    solve_a1,
    solve_a2,
    calc_obj_from_mat,
    calc_obj_from_vec,
    get_test_case_2_params,
    get_schedule_vec,
    _dev_patch_swap_infeas,
    mat3d_lp_sol_into_vec,
    vec_lp_sol_into_mat3d
)

In [25]:
# Константы
INF = float("inf")

In [6]:
async def exec_test_osp_blp(w, params, test_ctx):
    ctx = dict()
    x = await solve_osp_blp(w=w, ctx=ctx)
    obj = calc_obj_from_mat(w, x)
    x_vec = mat2d_lap_sol_into_vec(mat2d(x))
    return {
        "solver": "highs",
        "exec_time": ctx.get("exec_time"),
        "ts": int(datetime.datetime.now().timestamp()),
        "obj": obj,
        "x": x_vec,
    }
async def exec_inst_osp_blp(instance, test_ctx):
    params = instance["params"]
    w = create_cost_matrix_twscp(**params)
    return exec_test_osp_blp(w, params, test_ctx)

In [7]:
async def exec_test_osp_lp(w, params, test_ctx):
    ctx = dict()
    x_lp = await solve_lp(w=w, model_path="./model_lp.mzn", ctx=ctx)
    test_ctx["x_lp"] = x_lp
    obj = calc_obj_from_mat(w, x_lp)
    x_lp_vec = mat3d_lp_sol_into_vec(x_lp)
    
    return {
        "solver": "highs",
        "exec_time": ctx.get("exec_time"),
        "ts": int(datetime.datetime.now().timestamp()),
        "obj": obj,
        "x": x_lp_vec,
    }

In [8]:
async def exec_test_osp_lap_bnb_2d_v40__lc_arc(w, params, test_ctx):
    w2d = mat2d(w)
    ctx = {
        "params": params,
        "arc_branch_rule": "lowest_cost_arc",
        "tolerance_branch_rule": None,
    }
    _result = {
        "bnb_solver": "solve_osp_lap_bnb_2d_v4_0",
        "lap_solver": "lapjv",
    }
    try:
        x = await solve_osp_lap_bnb_2d_v4_0(w2d, ctx=ctx)
        if x:
            _result["obj"]            = ctx.get("obj")
            _result["n_tree_nodes"]   = ctx.get("n_tree_nodes", None)
            _result["n_lap_solved"]   = ctx.get("n_lap_solved", None)
            _result["n_cut_obj_ub"]   = ctx.get("n_cut_obj_ub", None)
            _result["n_cut_lb_ub"]    = ctx.get("n_cut_lb_ub", None)
            _result["n_cut_inf"]      = ctx.get("n_cut_inf", None)
            _result["n_cut_feasible"] = ctx.get("n_cut_feasible", None)
            _result["n_cut_no_opt"]   = ctx.get("n_cut_no_opt", None)
            _result["n_created_w"]    = ctx.get("n_created_w", None)
            _result["n_branching"]    = ctx.get("n_branching", None)
            _result["stop_reason"]    = ctx.get("stop_reason", None)
            _result["exec_time"]      = ctx.get("exec_time", None)
            _result["ts"]             = int(datetime.datetime.now().timestamp())
            _result["x"]              = ctx.get("x", None)
    except BaseException as e:
        _result["err"] = str(e)
    return _result
async def exec_inst_osp_lap_bnb_2d_v40__lc_arc(instance, test_ctx):
    params = instance["params"]
    w = create_cost_matrix_twscp(**params)
    return await exec_test_osp_lap_bnb_2d_v40__lc_arc(w, params, test_ctx)

In [9]:
async def exec_test_osp_lap_bnb_2d_v40__hc_arc(w, params, test_ctx):
    w2d = mat2d(w)
    ctx = {
        "params": params,
        "arc_branch_rule": "highest_cost_arc",
        "tolerance_branch_rule": None,
    }
    _result = {
        "bnb_solver": "solve_osp_lap_bnb_2d_v4_0",
        "lap_solver": "lapjv",
    }
    try:
        x = await solve_osp_lap_bnb_2d_v4_0(w2d, ctx=ctx)
        if x:
            _result["obj"]            = ctx.get("obj")
            _result["n_tree_nodes"]   = ctx.get("n_tree_nodes", None)
            _result["n_lap_solved"]   = ctx.get("n_lap_solved", None)
            _result["n_cut_obj_ub"]   = ctx.get("n_cut_obj_ub", None)
            _result["n_cut_lb_ub"]    = ctx.get("n_cut_lb_ub", None)
            _result["n_cut_inf"]      = ctx.get("n_cut_inf", None)
            _result["n_cut_feasible"] = ctx.get("n_cut_feasible", None)
            _result["n_cut_no_opt"]   = ctx.get("n_cut_no_opt", None)
            _result["n_created_w"]    = ctx.get("n_created_w", None)
            _result["n_branching"]    = ctx.get("n_branching", None)
            _result["stop_reason"]    = ctx.get("stop_reason", None)
            _result["exec_time"]      = ctx.get("exec_time", None)
            _result["ts"]             = int(datetime.datetime.now().timestamp())
            _result["x"]              = ctx.get("x", None)
    except BaseException as e:
        _result["err"] = str(e)
    return _result
async def exec_inst_osp_lap_bnb_2d_v40__hc_arc(instance, test_ctx):
    params = instance["params"]
    w = create_cost_matrix_twscp(**params)
    return await exec_test_osp_lap_bnb_2d_v40__hc_arc(w, params, test_ctx)

In [10]:
async def exec_test_osp_lap_bnb_2d_v40__1st_arc(w, params, test_ctx):
    w2d = mat2d(w)
    ctx = {
        "params": params,
        "arc_branch_rule": "first_available_arc",
        "tolerance_branch_rule": None,
    }
    _result = {
        "bnb_solver": "solve_osp_lap_bnb_2d_v4_0",
        "lap_solver": "lapjv",
    }
    try:
        x = await solve_osp_lap_bnb_2d_v4_0(w2d, ctx=ctx)
        if x:
            _result["obj"]            = ctx.get("obj")
            _result["n_tree_nodes"]   = ctx.get("n_tree_nodes", None)
            _result["n_lap_solved"]   = ctx.get("n_lap_solved", None)
            _result["n_cut_obj_ub"]   = ctx.get("n_cut_obj_ub", None)
            _result["n_cut_lb_ub"]    = ctx.get("n_cut_lb_ub", None)
            _result["n_cut_inf"]      = ctx.get("n_cut_inf", None)
            _result["n_cut_feasible"] = ctx.get("n_cut_feasible", None)
            _result["n_cut_no_opt"]   = ctx.get("n_cut_no_opt", None)
            _result["n_created_w"]    = ctx.get("n_created_w", None)
            _result["n_branching"]    = ctx.get("n_branching", None)
            _result["stop_reason"]    = ctx.get("stop_reason", None)
            _result["exec_time"]      = ctx.get("exec_time", None)
            _result["ts"]             = int(datetime.datetime.now().timestamp())
            _result["x"]              = ctx.get("x", None)
    except BaseException as e:
        _result["err"] = str(e)
    return _result
async def exec_inst_osp_lap_bnb_2d_v40__1st_arc(instance, test_ctx):
    params = instance["params"]
    w = create_cost_matrix_twscp(**params)
    return await exec_test_osp_lap_bnb_2d_v40__1st_arc(w, params, test_ctx)

In [11]:
async def exec_test_osp_lap_bnb_2d_v40__lc_arc_utol(w, params, test_ctx):
    w2d = mat2d(w)
    ctx = {
        "params": params,
        "arc_branch_rule": "lowest_cost_arc",
        "tolerance_branch_rule": "arc_upper_tolerance",
    }
    _result = {
        "bnb_solver": "solve_osp_lap_bnb_2d_v4_0",
        "lap_solver": "lapjv",
    }
    try:
        x = await solve_osp_lap_bnb_2d_v4_0(w2d, ctx=ctx)
        if x:
            _result["obj"]            = ctx.get("obj")
            _result["n_tree_nodes"]   = ctx.get("n_tree_nodes", None)
            _result["n_lap_solved"]   = ctx.get("n_lap_solved", None)
            _result["n_cut_obj_ub"]   = ctx.get("n_cut_obj_ub", None)
            _result["n_cut_lb_ub"]    = ctx.get("n_cut_lb_ub", None)
            _result["n_cut_inf"]      = ctx.get("n_cut_inf", None)
            _result["n_cut_feasible"] = ctx.get("n_cut_feasible", None)
            _result["n_cut_no_opt"]   = ctx.get("n_cut_no_opt", None)
            _result["n_created_w"]    = ctx.get("n_created_w", None)
            _result["n_branching"]    = ctx.get("n_branching", None)
            _result["stop_reason"]    = ctx.get("stop_reason", None)
            _result["exec_time"]      = ctx.get("exec_time", None)
            _result["ts"]             = int(datetime.datetime.now().timestamp())
            _result["x"]              = ctx.get("x", None)
    except BaseException as e:
        _result["err"] = str(e)
    return _result
async def exec_inst_osp_lap_bnb_2d_v40__lc_arc_utol(instance, test_ctx):
    params = instance["params"]
    w = create_cost_matrix_twscp(**params)
    return await exec_test_osp_lap_bnb_2d_v40__lc_arc_utol(w, params, test_ctx)

In [12]:
async def exec_test_osp_lap_bnb_2d_v40__lowest_utol(w, params, test_ctx):
    w2d = mat2d(w)
    ctx = {
        "params": params,
        "arc_branch_rule": "lowest_cost_arc",
        "tolerance_branch_rule": "lowest_upper_tolerance",
    }
    _result = {
        "bnb_solver": "solve_osp_lap_bnb_2d_v4_0",
        "lap_solver": "lapjv",
    }
    try:
        x = await solve_osp_lap_bnb_2d_v4_0(w2d, ctx=ctx)
        if x:
            _result["obj"]            = ctx.get("obj")
            _result["n_tree_nodes"]   = ctx.get("n_tree_nodes", None)
            _result["n_lap_solved"]   = ctx.get("n_lap_solved", None)
            _result["n_cut_obj_ub"]   = ctx.get("n_cut_obj_ub", None)
            _result["n_cut_lb_ub"]    = ctx.get("n_cut_lb_ub", None)
            _result["n_cut_inf"]      = ctx.get("n_cut_inf", None)
            _result["n_cut_feasible"] = ctx.get("n_cut_feasible", None)
            _result["n_cut_no_opt"]   = ctx.get("n_cut_no_opt", None)
            _result["n_created_w"]    = ctx.get("n_created_w", None)
            _result["n_branching"]    = ctx.get("n_branching", None)
            _result["stop_reason"]    = ctx.get("stop_reason", None)
            _result["exec_time"]      = ctx.get("exec_time", None)
            _result["ts"]             = int(datetime.datetime.now().timestamp())
            _result["x"]              = ctx.get("x", None)
    except BaseException as e:
        _result["err"] = str(e)
    return _result
async def exec_inst_osp_lap_bnb_2d_v40__lowest_utol(instance, test_ctx):
    params = instance["params"]
    w = create_cost_matrix_twscp(**params)
    return await exec_test_osp_lap_bnb_2d_v40__lowest_utol(w, params, test_ctx)

In [13]:
async def exec_test_wsrpt(w, params, test_ctx):
    _result = dict()
    try:
        t0 = time.process_time()
        s, obj = get_wsrpt_upper_bound(w, params)
        t1 = time.process_time()
        _result["exec_time"] = t1 - t0
        _result["obj"]       = obj
        _result["s"]         = s
        _result["ts"]        = int(datetime.datetime.now().timestamp())
    except BaseException as e:
        _result["err"] = str(e)
    return _result

In [14]:
async def exec_test_a1(w, params, test_ctx):
    _result = dict()
    x_lp = test_ctx["x_lp"]
    try:
        t0 = time.process_time()
        s = await solve_a1(w, x_lp, params, opts=None)
        t1 = time.process_time()
        _result["exec_time"] = t1 - t0
        _result["obj"]       = calc_obj_from_vec(w, s)
        _result["s"]         = s
    except BaseException as e:
        _result["err"] = str(e)
    return _result

In [15]:
async def exec_test_a2(w, params, test_ctx):
    _result = dict()
    x_lp = test_ctx["x_lp"]
    try:
        t0 = time.process_time()
        s = await solve_a2(w, x_lp, params, opts=None)
        t1 = time.process_time()
        _result["exec_time"] = t1 - t0
        _result["obj"]       = calc_obj_from_vec(w, s)
        _result["s"]         = s
        _result["ts"]        = int(datetime.datetime.now().timestamp());
    except BaseException as e:
        _result["err"] = str(e)
    return _result

In [16]:
# Ручной тест серии задач

_stats_table = []

async def main():
    n, p = 4, 4
    epochs = 1
    
    for _ in tqdm(range(epochs)):
        pbar = tqdm(total=6, leave=False)

        _stats_row = {}
        _stats_table.append(_stats_row)
        
        params = generate_test_case_params_twscp(n, p)
        w = create_cost_matrix_twscp(**params)

        _stats_row["params"] = params

        test_ctx = dict()
        
        _stats_row["osp_blp"]                 = await exec_test_osp_blp(w, params, test_ctx);                         pbar.update(1);
        _stats_row["osp_lap_bnb_1st_arc"]     = await exec_test_osp_lap_bnb_2d_v40__1st_arc(w, params, test_ctx);     pbar.update(1);
        _stats_row["osp_lap_bnb_hc_arc"]      = await exec_test_osp_lap_bnb_2d_v40__hc_arc(w, params, test_ctx);      pbar.update(1);
        _stats_row["osp_lap_bnb_lc_arc"]      = await exec_test_osp_lap_bnb_2d_v40__lc_arc(w, params, test_ctx);      pbar.update(1);
        _stats_row["osp_lap_bnb_lc_arc_utol"] = await exec_test_osp_lap_bnb_2d_v40__lc_arc_utol(w, params, test_ctx); pbar.update(1);
        _stats_row["osp_lap_bnb_lo_utol"]     = await exec_test_osp_lap_bnb_2d_v40__lowest_utol(w, params, test_ctx); pbar.update(1);

        pbar.clear()
        pbar.close()
# await main()

In [17]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [18]:
# Статистика из ручной серии задач

def main():
    _table = []
    _test_types = list(_stats_table[0].keys())
    _test_props = {
        "exec_time": -1,
        "obj": -1,
        "n_tree_nodes": -1,
        "n_lap_solved": -1,
        "n_cut_obj_ub": 0, "n_cut_lb_ub": 0, "n_cut_inf": 0, "n_cut_feasible": 0, "n_cut_no_opt": 0,
        "n_created_w": 0, "n_branching": 0,
    }
    _test_types.remove("params")
    for statrow in _stats_table:
        _row = {}
        _table.append(_row)
        _row["p"] = statrow["params"]["p"][0]
        _obj = nget(statrow, ["osp_blp", "obj"]) or nget(statrow, ["osp_lap_bnb_lc_arc", "obj"])
        for test_type in _test_types:
            _row[f"{test_type}.eq"] = (statrow[test_type]["obj"] == _obj)
            for prop in _test_props:
                defval = _test_props[prop]
                key = f"{test_type}.{prop}"
                val = nget(statrow, [test_type, prop], defval)
                val = defval if (val is None) else val
                _row[key] = val
    
    df = pd.DataFrame(_table)

    # df = df[
    #     True
    #     & (df["osp_lap_bnb_lc_arc_utol.n_tree_nodes"] > 1)
    #     & (df["osp_lap_bnb_lc_arc_utol.n_lap_solved"] > 1)
    #     & (df["osp_lap_bnb_lc_arc_utol.eq"] == 1)
    # ]

    # qcol = "osp_lap_bnb_lc_arc.n_lap_solved"
    # q0 = df[qcol].quantile(0.05)
    # q1 = df[qcol].quantile(0.95)
    # df = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

    print(df.shape)
    
    df_mean = df.mean()

    for test_type in _test_types:
        key = f"{test_type}.n_lap_solved"
        df_mean[f"{test_type}.min_n_lap_solved"] = df[key].min()
        df_mean[f"{test_type}.med_n_lap_solved"] = df[key].median()
        df_mean[f"{test_type}.max_n_lap_solved"] = df[key].max()

    df_mean = df_mean.sort_index()

    print(df_mean)
# main()

In [19]:
# Собрать тестовые задачи. Записать их на диск

async def main():
    _generated_ids = dict()

    async def is_uniq(inst, data_path=None):
        nonlocal _generated_ids
        
        if _generated_ids.get(inst["id"], None):
            return False
        
        params  = inst["params"]
        n       = len(params["p"])
        p       = params["p"][0]
        npkey   = f"n{n}p{p}"
        
        problem_instances_path = data_path or default_problem_instances_path
        
        _dirpath  = problem_instances_path + "/json/" + npkey
        _filepath = _dirpath + "/" + inst["id"] + ".json"
        
        _is_exists = await aiofiles.os.path.exists(_filepath)

        return (not _is_exists)

    _cases = [
        # {"n": 3, "p": 2, "e": 1000},
        # {"n": 3, "p": 3, "e": 1000},
        # {"n": 3, "p": 4, "e": 1000},
        # {"n": 3, "p": 5, "e": 1000},
        # {"n": 3, "p": 10, "e": 500},
        
        # {"n": 4, "p": 2, "e": 1000},
        # {"n": 4, "p": 3, "e": 1000},
        # {"n": 4, "p": 4, "e": 1000},
        # {"n": 4, "p": 5, "e": 100},
        # {"n": 4, "p": 10, "e": 500},
        
        # {"n": 5, "p": 2, "e": 1000},
        # {"n": 5, "p": 3, "e": 1000},
        # {"n": 5, "p": 4, "e": 100},
        # {"n": 5, "p": 10, "e": 500},

        # {"n": 10, "p": 2, "e": 500},
        # {"n": 10, "p": 3, "e": 500},
        # {"n": 10, "p": 4, "e": 500},
        # {"n": 10, "p": 5, "e": 500},
        # {"n": 10, "p": 10, "e": 50},

        # {"n": 25, "p": 2, "e": 500},
        # {"n": 25, "p": 3, "e": 250},
        # {"n": 25, "p": 4, "e": 50},
        # {"n": 25, "p": 5, "e": 50},
        # {"n": 25, "p": 10, "e": 50},

        # {"n": 50, "p": 2, "e": 50},
        # {"n": 50, "p": 3, "e": 50},
        # {"n": 50, "p": 4, "e": 50},
        # {"n": 50, "p": 5, "e": 50},
        # {"n": 50, "p": 10, "e": 50},

        # {"n": 100, "p": 2, "e": 10},
        # {"n": 100, "p": 3, "e": 10},
        # {"n": 100, "p": 4, "e": 10},
        # {"n": 100, "p": 5, "e": 10},
        # {"n": 100, "p": 10, "e": 10},
    ]

    for _case in tqdm(_cases):
        n = _case["n"]
        p = _case["p"]
        e = _case["e"]

        for instance_i in tqdm(range(e), leave=False):
            # Повторять пока не соберется решаемый пример
            _safe_iterator = range(10)
            
            for _ in _safe_iterator: 
                params = generate_test_case_params_twscp(n, p)
                pid    = create_problem_instance_id({ "params": params })

                _instance = {
                    "id": pid,
                    "params": params,
                    "results": {},
                }

                # Неуникальный экземпляр задачи -- пропустить
                if (await is_uniq(_instance)) == False:
                    continue

                _generated_ids[pid] = 1

                await write_instance_into_disk(_instance)

                break
            else:
                print("!safe_counter")
                print(params)
                return

            # end _safe_iterator
        # end instance_i

# await main()

In [20]:
# # Решить тестовые задачи. Записать результаты на диск

# async def main():
#     pil = PILStd(default_problem_instances_path + "/json")

#     for instance in tqdm(pil):
#         params = instance["params"]
#         n = len(params["p"])
#         p = params["p"][0]

#         w = create_cost_matrix_twscp(**params)

#         tests = {
#             "osp_blp"                : { "test_func": exec_test_osp_blp },
#             "osp_lap_bnb_1st_arc"    : { "test_func": exec_test_osp_lap_bnb_2d_v40__1st_arc },
#             "osp_lap_bnb_lc_arc"     : { "test_func": exec_test_osp_lap_bnb_2d_v40__lc_arc },
#             "osp_lap_bnb_hc_arc"     : { "test_func": exec_test_osp_lap_bnb_2d_v40__hc_arc },
#             "osp_lap_bnb_lc_arc_utol": { "test_func": exec_test_osp_lap_bnb_2d_v40__lc_arc_utol },
#             "osp_lap_bnb_lo_utol"    : { "test_func": exec_test_osp_lap_bnb_2d_v40__lowest_utol },
#         }

#         _awaits = []
#         _keys = []
        
#         for a in tests:
#             if a not in instance["results"]:
#                 func = tests[a]["test_func"]
#                 _keys.append(a)
#                 _awaits.append(func(w, params))

#         if not _awaits:
#             continue

#         _awaits_res = await asyncio.gather(*_awaits)

#         for i, res in enumerate(_awaits_res):
#             instance["results"][_keys[i]] = res

#         await write_instance_into_disk(instance)
# # await main()

In [21]:
# Решить тестовые задачи. Записать результаты на диск

def _exec_test_func_dec(func):
    async def _wrap(w, params, test_ctx, instance, test_name):
        if test_name not in instance["results"]:
            res = await func(w, params, test_ctx)
            instance["results"][test_name] = res
            test_ctx["results_updated"] = 1
    return _wrap

async def _exec_lp_group_test_case(w, params, test_ctx, instance, test_name):
    for k in ["lp", "a1", "a2"]:
        if (k not in instance["results"]):
            instance["results"]["lp"] = await exec_test_osp_lp(w, params, test_ctx)
            instance["results"]["a1"] = await exec_test_a1(w, params, test_ctx)
            instance["results"]["a2"] = await exec_test_a2(w, params, test_ctx)
            test_ctx["results_updated"] = 1
            return

async def main():
    pil = PILStd(default_problem_instances_path + "/json")
    pil_iter = iter(pil)

    counter = 0
    
    async def _exec():
        nonlocal counter

        while True:
            try:
                instance = next(pil_iter)
            except StopIteration:
                return
            
            if counter % 20 == 0:
                clear_output()
            
            params = instance["params"]
            n = len(params["p"])
            p = params["p"][0]

            if not (n == 5 and p == 5):
                continue
            
            print(f"{counter}: id={instance['id']}, n={n}, p={p}")
            
            w = create_cost_matrix_twscp(**params)
            
            tests = [
                { "name": "wsrpt",      "func": _exec_test_func_dec(exec_test_wsrpt)   },
                { "name": "_lp_group",  "func": _exec_lp_group_test_case               },
                { "name": "osp_blp",    "func": _exec_test_func_dec(exec_test_osp_blp) },
            ]

            # if (n <= 5) and (n*p <= 20):
            if True:
                tests.extend([
                    # { "name": "osp_lap_bnb_1st_arc",     "func": _exec_test_func_dec(exec_test_osp_lap_bnb_2d_v40__1st_arc)     },
                    { "name": "osp_lap_bnb_lc_arc",      "func": _exec_test_func_dec(exec_test_osp_lap_bnb_2d_v40__lc_arc)      },
                    # { "name": "osp_lap_bnb_hc_arc",      "func": _exec_test_func_dec(exec_test_osp_lap_bnb_2d_v40__hc_arc)      },
                    { "name": "osp_lap_bnb_lc_arc_utol", "func": _exec_test_func_dec(exec_test_osp_lap_bnb_2d_v40__lc_arc_utol) },
                    { "name": "osp_lap_bnb_lo_utol",     "func": _exec_test_func_dec(exec_test_osp_lap_bnb_2d_v40__lowest_utol) },
                ])
            
            test_ctx = {
                "results_updated": 0,
            }
            
            _awaits = []
            
            for test in tests:
                _awaits.append(
                    test["func"](
                        w=w,
                        params=params,
                        test_ctx=test_ctx,
                        instance=instance,
                        test_name=test["name"]
                    )
                )
            
            if _awaits:
                await asyncio.gather(*_awaits)
            
            if test_ctx["results_updated"]:
                await write_instance_into_disk(instance)
            
            counter = counter + 1

    await asyncio.gather(*[
        _exec()
        for _ in range(async_threads_n)
    ])

# await main()

In [24]:
# Ручной тест одной случайно задачи
async def main():
    n, p = 3, 3
    params = generate_test_case_params_twscp(n, p)
    
    print("params = ", params)

    np = n*p
    w   = create_cost_matrix_twscp(**params)
    w2d = mat2d(w)

    print()
    if np < 50:
        print("w =")
        print_osp_matrix(w, jsep=" ")

    print()
    print("LP")
    x_lp = await solve_lp(w, "./model_lp.mzn")
    if np < 50:
        print("x =")
        print_osp_matrix(deep_round(x_lp, 2), jsep=" ")
    print("obj =", calc_obj_from_mat(w, x_lp))
    
    print()
    print("BLP")
    x_blp = await solve_osp_blp(w)
    if np < 50:
        print("x =")
        print_osp_matrix(x_blp, jsep=" ")
    print(get_schedule_vec(x_blp))
    print("obj =", calc_obj_from_mat(w, x_blp))

    print()
    print("BnB")
    ctx = { "params": params, "arc_branch_rule": "lowest_cost_arc" }
    x_bnb = await solve_osp_lap_bnb_2d_v4_0(w2d, ctx)
    print("x =")
    print_osp_matrix(x_bnb)
    print("obj =", calc_obj_from_mat(w2d, x_bnb))

    print()
    print("WSRPT")
    print("s =")
    wsrpt = get_wsrpt_upper_bound(w, params)
    print(wsrpt)

    print()
    print("LAP")
    x_lap = solve_lap_scipy(w2d)
    x_lap3d = mat3d(x_lap, params["p"])
    if np < 50:
        print("x =")
        print_osp_matrix(x_lap3d, jsep=" ")
    print(get_schedule_vec(x_lap3d))
    print("obj =", calc_obj_from_mat(w2d, x_lap))

    print()
    print("SWAP_PATCH")
    x_lap_patched = _dev_patch_swap_infeas(copy.deepcopy(x_lap), params)
    print("obj = ", calc_obj_from_mat(w2d, x_lap_patched))

    print()
    print("A1")
    s_a1 = await solve_a1(w, x_lp, params)
    print("s =", s_a1)
    print("obj =", calc_obj_from_vec(w, s_a1))

    print()
    print("A2")
    s_a2 = await solve_a2(w, x_lp, params)
    print("s =", s_a2)
    print("obj =", calc_obj_from_vec(w, s_a2))
await main()

params =  {'p': [3, 3, 3], 'w': [27, 17, 25], 'd1': [9, 4, 6], 'd2': [9, 6, 6], 's1': [4, 4, 3], 's2': [5, 4, 3], 'r': [3, 1, 2]}

w =
[59049, 59049,    27,     0,     0,    27,    54, 59049, 59049]
[59049, 59049, 59049,     0,     0,     0,     0,     0, 59049]
[59049, 59049, 59049, 59049,   108,    81,    54,    27,     0]
 
[   51,    34,    17,     0,    17,    34,    51, 59049, 59049]
[59049,     0,     0,     0,     0,     0,     0,     0, 59049]
[59049, 59049,    17,     0,     0,     0,    17,    34,    51]
 
[59049,    25,     0,    25,    50,    75,   100, 59049, 59049]
[59049, 59049,     0,     0,     0,     0,     0,     0, 59049]
[59049, 59049, 59049,    50,    25,     0,    25,    50,    75]
 

LP
x =
[0.0, 0.0,-0.0,0.67,0.33,-0.0,-0.0,-0.0,-0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,-0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
 
[1.0, 0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0]
[0.0, 1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0]
[0.0, 0.0, 0.0, 0.0,0.67,-0.0,0.33,-0.0,-0.